<a href="https://colab.research.google.com/github/harshinshah29/133AProjectGroup17/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [2]:
# FOR LOWERCASE P
try:
  df = pd.read_csv('/content/drive/MyDrive/ECE 133A project/Datasets/hotel_booking.csv')
  df.info()
except Exception as e:
  df = pd.read_csv('/content/drive/MyDrive/ECE 133A Project/Datasets/hotel_booking.csv')
  df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78290 entries, 0 to 78289
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   is_canceled                     78290 non-null  int64  
 1   hotel                           78290 non-null  object 
 2   lead_time                       78290 non-null  int64  
 3   arrival_date_month              78290 non-null  object 
 4   stays_in_weekend_nights         78290 non-null  int64  
 5   stays_in_week_nights            78290 non-null  int64  
 6   adults                          78290 non-null  int64  
 7   children                        78287 non-null  float64
 8   babies                          78290 non-null  int64  
 9   meal                            78290 non-null  object 
 10  country                         78290 non-null  object 
 11  previous_cancellations          78290 non-null  int64  
 12  previous_bookings_not_canceled  

In [3]:
#FOR UPPERCASE P
df = pd.read_csv('/content/drive/MyDrive/ECE 133A Project/Datasets/hotel_booking.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78290 entries, 0 to 78289
Data columns (total 24 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   is_canceled                     78290 non-null  int64  
 1   hotel                           78290 non-null  object 
 2   lead_time                       78290 non-null  int64  
 3   arrival_date_month              78290 non-null  object 
 4   stays_in_weekend_nights         78290 non-null  int64  
 5   stays_in_week_nights            78290 non-null  int64  
 6   adults                          78290 non-null  int64  
 7   children                        78287 non-null  float64
 8   babies                          78290 non-null  int64  
 9   meal                            78290 non-null  object 
 10  country                         78290 non-null  object 
 11  previous_cancellations          78290 non-null  int64  
 12  previous_bookings_not_canceled  

In [4]:
#dropping unnecessary fields
df = df.drop(columns=['name', 'email', 'phone-number'])

In [5]:
#one hot encoding
categorical_cols = ['hotel', 'arrival_date_month', 'meal', 'country', 'reserved_room_type', 'deposit_type', 'customer_type']
ohe = OneHotEncoder(drop='first', sparse_output=False)
df_encoded = pd.DataFrame(ohe.fit_transform(df[categorical_cols]))
df_encoded.columns = ohe.get_feature_names_out(categorical_cols)
df.drop(columns=categorical_cols, inplace=True)
df = pd.concat([df, df_encoded], axis=1)

In [6]:
#handling missing values
df['children'].fillna(0, inplace=True)

In [7]:
#augment features
df['total_stay'] = df['stays_in_weekend_nights'] + df['stays_in_week_nights']

df['total_guests'] = df['adults'] + df['children'] + df['babies']

df['total_previous_bookings'] = df['previous_cancellations'] + df['previous_bookings_not_canceled']

df['total_cost'] = df['adr'] / df['total_stay']

df['adults_ratio'] = df['adults'] / df['total_guests']

df['cars_to_guests'] = df['required_car_parking_spaces'] / df['total_guests']

df['guests_to_cost'] = df['total_guests'] + df['total_cost']

In [8]:
#drop nonsensical entries
df = df[df['total_stay'] != 0]
df = df[df['total_guests'] != 0]

In [9]:
# Check for NaN or inf values
nan_mask = df.isna()

inf_mask = df.applymap(np.isinf)

nan_or_inf_mask = nan_mask | inf_mask

df[nan_or_inf_mask.any(axis=1)]

,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,...,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,total_stay,total_guests,total_previous_bookings,total_cost,adults_ratio,cars_to_guests,guests_to_cost


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77640 entries, 0 to 78289
Data columns (total 55 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   is_canceled                     77640 non-null  int64  
 1   lead_time                       77640 non-null  int64  
 2   stays_in_weekend_nights         77640 non-null  int64  
 3   stays_in_week_nights            77640 non-null  int64  
 4   adults                          77640 non-null  int64  
 5   children                        77640 non-null  float64
 6   babies                          77640 non-null  int64  
 7   previous_cancellations          77640 non-null  int64  
 8   previous_bookings_not_canceled  77640 non-null  int64  
 9   booking_changes                 77640 non-null  int64  
 10  days_in_waiting_list            77640 non-null  int64  
 11  adr                             77640 non-null  float64
 12  required_car_parking_spaces     

## Standardizing Features

In [11]:
numerical_cols = ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'babies', 'children', 'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'days_in_waiting_list', 'adr','required_car_parking_spaces','total_of_special_requests', 'total_stay', 'total_guests', 'total_previous_bookings','total_cost','adults_ratio', 'cars_to_guests', 'guests_to_cost' ]
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])
df.describe()

,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,...,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,total_stay,total_guests,total_previous_bookings,total_cost,adults_ratio,cars_to_guests,guests_to_cost
count,77640.000000,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,...,77640.000000,77640.000000,77640.000000,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04,7.764000e+04
mean,0.408836,-1.885263e-17,9.142609e-17,-6.225486e-17,7.358016e-17,-4.804675e-17,-5.033469e-17,-4.484363e-18,-1.972205e-17,-3.349545e-17,...,0.005165,0.730848,0.223712,-1.830352e-17,-2.037182e-16,-9.517832e-18,7.074311e-17,-1.307970e-15,1.038725e-16,4.420301e-17
std,0.491622,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,...,0.071682,0.443522,0.416734,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00
min,0.000000,-9.658937e-01,-9.013631e-01,-1.325806e+00,-3.007659e+00,-2.419117e-01,-8.281927e-02,-1.232062e-01,-1.003800e-01,-3.432837e-01,...,0.000000,0.000000,0.000000,-9.404342e-01,-1.273818e+00,-1.362777e-01,-1.093208e+00,-8.817322e+00,-2.429311e-01,-1.099351e+00
25%,0.000000,-8.164690e-01,-9.013631e-01,-7.855467e-01,2.568190e-01,-2.419117e-01,-8.281927e-02,-1.232062e-01,-1.003800e-01,-3.432837e-01,...,0.000000,0.000000,0.000000,-5.388430e-01,7.996452e-02,-1.362777e-01,-5.515229e-01,2.605143e-01,-2.429311e-01,-5.502695e-01
50%,0.000000,-3.330363e-01,1.139700e-01,-2.452876e-01,2.568190e-01,-2.419117e-01,-8.281927e-02,-1.232062e-01,-1.003800e-01,-3.432837e-01,...,0.000000,1.000000,0.000000,-1.372517e-01,7.996452e-02,-1.362777e-01,-2.650676e-01,2.605143e-01,-2.429311e-01,-2.632497e-01
75%,1.000000,5.283528e-01,1.129303e+00,2.949714e-01,2.568190e-01,-2.419117e-01,-8.281927e-02,-1.232062e-01,-1.003800e-01,-3.432837e-01,...,0.000000,1.000000,0.000000,2.643395e-01,7.996452e-02,-1.362777e-01,2.749804e-01,2.605143e-01,-2.429311e-01,2.758485e-01
max,1.000000,5.265993e+00,1.534397e+01,2.028456e+01,8.676548e+01,2.676704e+01,9.503683e+01,2.913323e+01,4.121099e+01,2.891497e+01,...,1.000000,1.000000,1.000000,2.114708e+01,7.183042e+01,3.723407e+01,1.338722e+02,2.605143e-01,1.303335e+01,1.337001e+02


In [12]:
import numpy as np
# Check for NaN or inf values
nan_mask = df.isna()


inf_mask = df.applymap(np.isinf)

nan_or_inf_mask = nan_mask | inf_mask

df[nan_or_inf_mask.any(axis=1)]

,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,...,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,total_stay,total_guests,total_previous_bookings,total_cost,adults_ratio,cars_to_guests,guests_to_cost


## SVD

In [13]:
import numpy as np
U, s, Vt = np.linalg.svd(df, full_matrices=False)

In [14]:
first_component = np.abs(Vt[0, :])

top_features_indices = np.argsort(first_component)[-8:]

top_feature_names = df.columns[top_features_indices]

top_feature_names

Index(['children', 'lead_time', 'adr', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'total_stay', 'guests_to_cost', 'total_cost'],
      dtype='object')

# K Means Algorithm

In [ ]:
# Finding Best K Value

sil = []
kmax = 15

# dissimilarity not be defined for a single cluster so minimum number of clusters is 2
for k in range(2, kmax+1):
  kmeans = KMeans(n_clusters = k).fit(df)
  labels = kmeans.labels_
  sil.append(silhouette_score(df, labels, metric = 'euclidean'))


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KeyboardInterrupt: 

Exception ignored in: 'sklearn.cluster._k_means_common._relocate_empty_clusters_dense'
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/numpy/core/multiarray.py", line 346, in where
    @array_function_from_c_func_and_dispatcher(_multiarray_umath.where)
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
k_opt = sil.index(max(sil)) + 2

k_values = np.arange(2,kmax + 1,1)

plt.plot(k_values,sil)
plt.ylabel("Silhouette Score")
plt.xlabel("K value")
plt.show()

print("Best K Value: ", k_opt)
print("Best Silhouette Score: ", max(sil))

In [ ]:
from yellowbrick.cluster import KElbowVisualizer

# Initialize the KElbowVisualizer with the KMeans estimator and a range of K values
Elbow_M = KElbowVisualizer(KMeans(), k=kmax)
# Fit the visualizer
Elbow_M.fit(df)
# Display the Elbow Method plot
Elbow_M.show()

In [ ]:
# Fitting Dataset to K Means

kmeans = KMeans(n_clusters=k_opt, random_state=0, n_init="auto").fit(df)

kmeans.labels_

# Correlation Matrix

In [15]:
# calculate de-meaned vector
def de_mean_vector(vec):
  n = len(vec)
  avg = sum(vec)/n
  avg_vec = [avg] * n
  return vec - avg_vec

In [16]:
# find correlation coefficient
def correlation_coefficient(A, B):
  demeaned_a, demeaned_b = de_mean_vector(A), de_mean_vector(B)
  return (np.dot(demeaned_a, demeaned_b) / (np.linalg.norm(demeaned_a) * np.linalg.norm(demeaned_b)))

In [17]:
df.head()

,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,...,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,total_stay,total_guests,total_previous_bookings,total_cost,adults_ratio,cars_to_guests,guests_to_cost
0,0,-0.930735,0.113970,-0.245288,0.256819,-0.241912,-0.082819,-0.123206,-0.10038,-0.343284,...,0.0,1.0,0.0,-0.137252,0.079965,-0.136278,-0.452497,0.260514,-0.242931,-0.450437
1,1,0.545932,-0.901363,-0.245288,-1.375420,-0.241912,-0.082819,-0.123206,-0.10038,-0.343284,...,0.0,0.0,1.0,-0.538843,-1.273818,-0.136278,0.109792,0.260514,-0.242931,0.086166
2,0,-0.930735,1.129303,-0.785547,-1.375420,-0.241912,-0.082819,-0.123206,-0.10038,-0.343284,...,0.0,1.0,0.0,-0.137252,-1.273818,-0.136278,-0.535800,0.260514,-0.242931,-0.558589
3,1,-0.368195,-0.901363,-0.245288,0.256819,-0.241912,-0.082819,-0.123206,-0.10038,-0.343284,...,0.0,0.0,1.0,-0.538843,0.079965,-0.136278,-1.077264,0.260514,-0.242931,-1.074393
4,1,0.343769,1.129303,0.835231,1.889058,-0.241912,-0.082819,-0.123206,-0.10038,-0.343284,...,0.0,1.0,0.0,1.067522,1.433747,-0.136278,-0.378775,0.260514,-0.242931,-0.351851


In [18]:
n_cols = len(df.columns)
correlation_matrix = np.zeros(shape=(n_cols, n_cols))

for i in range(n_cols):
  for j in range(i+1, n_cols):
    A, B = df[df.columns[i]], df[df.columns[j]]
    rho = correlation_coefficient(A, B)
    correlation_matrix[i][j] = rho
    correlation_matrix[j][i] = rho

# correlation coefficient of a column with itself will equal 1
for i in range(n_cols):
  correlation_matrix[i][i] = 1

  print("Entries with correlation coefficient >= 0.7:")
for i in range(n_cols):
    for j in range(i+1, n_cols):
        if correlation_matrix[i][j] >= 0.6:
            print(f"{df.columns[i]} and {df.columns[j]}: {correlation_matrix[i][j]}")

correlation_matrix

<ipython-input-16-23c1a24b8acb>:4: RuntimeWarning: invalid value encountered in scalar divide
  return (np.dot(demeaned_a, demeaned_b) / (np.linalg.norm(demeaned_a) * np.linalg.norm(demeaned_b)))


KeyboardInterrupt: 

## Linear Model

In [19]:
# Define features and target
X = df.drop(columns=['is_canceled'])
y = df['is_canceled']
